# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Workspace and Experiment
ws = Workspace.from_config()

experiment_name = 'hyperdrive_experiment'
experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Creating a Compute Cluster specific for HyperDrive Run
cluster_name = "hyperdrive-cluster"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_V2', 
        max_nodes=4
    )

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Specification of an early termination policy
early_termination_policy = BanditPolicy(
    evaluation_interval=1,
    slack_factor = 0.1,  
    delay_evaluation=4
)

# Specify parameter sampler
param_sampling = RandomParameterSampling( 
    {
    "--C" : uniform(0.1,1),
    "--max_iter" : choice(50, 100, 150, 200)
    }
)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(
    name='sklearn-env', 
    file_path='conda_dependencies.yml'
)

# Create your estimator and hyperdrive config
if "training" not in os.listdir():
        os.mkdir("./training")

estimator = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_cluster,
    environment=sklearn_env
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(
    config=hyperdrive_run_config,
    show_output=True
)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get the best model
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()
hyperdrive_parameter_values = best_hyperdrive_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Accuracy: ', best_hyperdrive_run_metrics['Accuracy'])
print('\n Hyperparameters: ', hyperdrive_parameter_values)

In [ ]:
# Save the best model as joblib and pkl file
joblib.dump(best_hyperdrive_run, 'best-hyperdrive-model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
## See for more details: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb

model = Model.register(
    workspace=ws,
    model_name = "best-hyperdrive-model", 
    model_path = './best-hyperdrive-model.pkl')

#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'conda_dependencies.yml')
print(model.name, model.id, model.version, sep='\t')

inference_config = InferenceConfig(
    entry_script='score_hyperdrive.py', 
    environment=sklearn_env
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    auth_enabled=True,
    tags = {'name':'failure Prediction'},
    description='Heart failure prediction model'
)

service = model.deploy(
    workspace=ws,
    name="hyperdrive-deploy",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)

In [ ]:
# Enable Logging and check state
service.update(enable_app_insights=True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# URL for the web service
######## ToDo:
scoring_uri = ''
# If the service is authenticated, set the key or token
######## ToDo:
key = ''

data = {
  "data": [
            "Inputs":{
                "data": [
                    {
                        'age': 75.0, 
                        'anaemia': 0.0, 
                        'creatinine_phosphokinase': 582.0,
                        'diabetes': 0.0,
                        'ejection_fraction': 20.0,
                        'high_blood_pressure': 1.0,
                        'platelets': 265000.0,
                        'serum_creatinine': 1.9,
                        'serum_sodium': 130.0,
                        'sex': 1.0,
                        'smoking': 0.0,
                        'time': 4.0          
                    }
                ]   
            },
            "GlobalParameters": {
            "method": "predict"
            }
        ]

}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(
    scoring_uri, 
    input_data, 
    headers=headers
    )
    
print("prediction ", resp.json())
# print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print the logs of the web service
print(service.get_logs())

# Delete the service
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

